In [1]:
!pip install transformers datasets
!pip install accelerate


In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
from google.colab import files
# uploaded = files.upload("/content/custom_data.txt")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Add special tokens if needed
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
def load_dataset(file_path, tokenizer, block_size=128):
    from transformers import TextDataset
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

def get_data_collator(tokenizer):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # GPT-2 is not a masked language model
    )


In [12]:
training_args = TrainingArguments(
    output_dir='./gpt2-finetuned',
    overwrite_output_dir=True,
    num_train_epochs=3,      # Increase if needed
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    prediction_loss_only=True,
    remove_unused_columns=False,
    report_to="none", # Disable wandb logging
)

In [13]:


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=3, training_loss=2.666905085245768, metrics={'train_runtime': 46.7014, 'train_samples_per_second': 0.064, 'train_steps_per_second': 0.064, 'total_flos': 195969024000.0, 'train_loss': 2.666905085245768, 'epoch': 3.0})

In [14]:
model.save_pretrained('./gpt2-finetuned')
tokenizer.save_pretrained('./gpt2-finetuned')


('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json')

In [15]:
from transformers import pipeline

generator = pipeline('text-generation', model='./gpt2-finetuned', tokenizer=tokenizer)

prompt = "Once upon a time"
outputs = generator(prompt, max_length=100, num_return_sequences=1)

print(outputs[0]['generated_text'])


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Once upon a time, a strange thing happened at the very heart of an ancient world, and one who has been there before vanished.

A mysterious voice called the Ghost spoke of something called the Gremlins, a group of people who made up the ancient inhabitants of the world.

They were a group of creatures who came from a distant land, and lived in a city called the Glowspire where the Gremlins ruled.

The Gremlins were one of the few living beings who did not use magic, and had lived in the glowspire for an entire hundred,000 years.

The Gremlins had been a race of the undead, who had been driven from their homeland by the power of the undead. The Gremlins were now the most feared group of undead in the world, and had been known to hunt down and kill anyone they came across. At one point, a single Gremlin had turned their back on the world and turned to the spirit world, where they would hunt down and kill anyone they found.

The Gremlins had been brought to the glowspire by a creature kno